In [1]:
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
# ✅ Set dataset path
train_dir = "C:/Users/adith/OneDrive/Desktop/ASL/Data"

# ✅ Check class distribution
class_counts = {folder: len(os.listdir(os.path.join(train_dir, folder))) for folder in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, folder))}
print("Dataset Class Distribution:", class_counts)


Dataset Class Distribution: {'A': 150, 'B': 150, 'C': 150, 'D': 150, 'E': 150, 'F': 150, 'G': 150, 'H': 150, 'I': 150, 'J': 150, 'K': 150, 'L': 150, 'M': 150, 'N': 150, 'O': 150, 'P': 150, 'Q': 150, 'R': 150, 'S': 150, 'T': 150, 'U': 150, 'V': 150, 'W': 150, 'X': 150, 'Y': 150, 'Z': 150}


In [3]:
# ✅ ImageDataGenerator with Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  
    rotation_range=15,   
    width_shift_range=0.2,  
    height_shift_range=0.2,  
    zoom_range=0.2,  
    horizontal_flip=True,  
    validation_split=0.2  
)


In [4]:
# ✅ Load training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    color_mode="grayscale",  # Matches dataset format
    class_mode="categorical",
    subset="training"
)

Found 3120 images belonging to 26 classes.


In [5]:
# ✅ Load validation data
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    color_mode="grayscale",  # Matches dataset format
    class_mode="categorical",
    subset="validation"
)

# ✅ Print the class labels
print("Class Labels Mapping:", train_generator.class_indices)

Found 780 images belonging to 26 classes.
Class Labels Mapping: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}


In [6]:

# ✅ Improved CNN Model
model = Sequential([
    # First Conv Block
    Conv2D(32, (3, 3), padding='same', input_shape=(128, 128, 1)),  
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Second Conv Block
    Conv2D(64, (3, 3), padding='same'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Third Conv Block
    Conv2D(128, (3, 3), padding='same'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Fourth Conv Block
    Conv2D(256, (3, 3), padding='same'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Fifth Conv Block (Extra for better learning)
    Conv2D(512, (3, 3), padding='same'),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Global Average Pooling (Better than Flatten for generalization)
    GlobalAveragePooling2D(),

    # Fully Connected Layer
    Dense(512, activation='relu'),
    Dropout(0.5),  

    # Output Layer
    Dense(len(train_generator.class_indices), activation='softmax')
])



In [7]:
# ✅ Compile Model with lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# ✅ Train the Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,  
    verbose=1
)

Epoch 1/20
98/98 [==============================] - 63s 620ms/step - loss: 2.6629 - accuracy: 0.2115 - val_loss: 3.7251 - val_accuracy: 0.0346
Epoch 2/20
98/98 [==============================] - 57s 577ms/step - loss: 1.8252 - accuracy: 0.4404 - val_loss: 5.2201 - val_accuracy: 0.0385
Epoch 3/20
98/98 [==============================] - 60s 610ms/step - loss: 1.3398 - accuracy: 0.6038 - val_loss: 5.9317 - val_accuracy: 0.0385
Epoch 4/20
98/98 [==============================] - 58s 592ms/step - loss: 1.0167 - accuracy: 0.7032 - val_loss: 5.4241 - val_accuracy: 0.0718
Epoch 5/20
98/98 [==============================] - 57s 578ms/step - loss: 0.7894 - accuracy: 0.7804 - val_loss: 4.1517 - val_accuracy: 0.0846
Epoch 6/20
98/98 [==============================] - 58s 587ms/step - loss: 0.6116 - accuracy: 0.8285 - val_loss: 2.1201 - val_accuracy: 0.3795
Epoch 7/20
98/98 [==============================] - 57s 579ms/step - loss: 0.4794 - accuracy: 0.8756 - val_loss: 1.2245 - val_accuracy: 0.5705

In [8]:
model.save("my_model.h5")

c:\Users\adith\OneDrive\Desktop\ASL\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
